In [77]:
"""
Implementation of a simple multivalent binding model.
"""

import numpy as np
import pandas as pd
import pathlib
from os.path import dirname, join
from scipy.optimize import root
from scipy.optimize import minimize


path_here = pathlib.Path().absolute()
KxStarP = 1.126e-12
x0 = []


def getKxStar():
    return KxStarP


def Req_func2(Req, L0, KxStar, Rtot, Cplx, Ctheta, Kav):
    Psi = Req * Kav * KxStar
    Psi = np.pad(Psi, ((0, 0), (0, 1)), constant_values=1)
    Psirs = np.sum(Psi, axis=1).reshape(-1, 1)
    Psinorm = (Psi / Psirs)[:, :-1]

    Rbound = L0 / KxStar * np.sum(Ctheta.reshape(-1, 1) * np.dot(Cplx, Psinorm) * np.exp(np.dot(Cplx, np.log1p(Psirs - 1))), axis=0)
    return Req + Rbound - Rtot


def polyc(L0, KxStar, Rtot, Cplx, Ctheta, Kav):
    """
    The main function to be called for multivalent binding
    :param L0: concentration of ligand complexes
    :param KxStar: Kx for detailed balance correction
    :param Rtot: numbers of each receptor on the cell
    :param Cplx: the monomer ligand composition of each complex
    :param Ctheta: the composition of complexes
    :param Kav: Ka for monomer ligand to receptors
    :return:
        Lbound: a list of Lbound of each complex
        Rbound: a list of Rbound of each kind of receptor
    """
    # Consistency check
    Kav = np.array(Kav)
    assert Kav.ndim == 2
    Rtot = np.array(Rtot, dtype=np.float64)
    assert Rtot.ndim == 1
    Cplx = np.array(Cplx)
    assert Cplx.ndim == 2
    Ctheta = np.array(Ctheta)
    assert Ctheta.ndim == 1

    assert Kav.shape[0] == Cplx.shape[1]
    assert Kav.shape[1] == Rtot.size
    assert Cplx.shape[0] == Ctheta.size
    Ctheta = Ctheta / np.sum(Ctheta)

    # Solve Req
    lsq = root(Req_func2, Rtot, method="lm", args=(L0, KxStar, Rtot, Cplx, Ctheta, Kav), options={"maxiter": 3000})
    assert lsq["success"], "Failure in rootfinding. " + str(lsq)
    Req = lsq["x"].reshape(1, -1)

    # Calculate the results
    Psi = np.ones((Kav.shape[0], Kav.shape[1] + 1))
    Psi[:, : Kav.shape[1]] *= Req * Kav * KxStar
    Psirs = np.sum(Psi, axis=1).reshape(-1, 1)
    Psinorm = (Psi / Psirs)[:, :-1]

    Rbound = L0 / KxStar * Ctheta.reshape(-1, 1) * np.dot(Cplx, Psinorm) * np.exp(np.dot(Cplx, np.log1p(Psirs - 1)))
    assert Rbound.shape[0] == len(Ctheta)
    assert Rbound.shape[1] == len(Rtot)
    return Rbound


def cytBindingModel(Kx, Cplx, doseVec, cellType, animal, slopes):
    """Runs binding model for a given mutein, valency, dose, and cell type."""
    recQuantDF = pd.read_csv(join(path_here, "data/RecQuant.csv"))
    recCount = np.ravel([recQuantDF.loc[(recQuantDF.Receptor == "IL4Ra") & (recQuantDF["Cell"] == cellType) & (recQuantDF["Animal"] == animal)].Amount.values,
                             recQuantDF.loc[(recQuantDF.Receptor == "Gamma") & (recQuantDF["Cell"] == cellType) & (recQuantDF["Animal"] == animal)].Amount.values,
                             recQuantDF.loc[(recQuantDF.Receptor == "IL13Ra") & (recQuantDF["Cell"] == cellType) & (recQuantDF["Animal"] == animal)].Amount.values])
    print(cellType)
    print(recCount)

    if doseVec.size == 1:
        doseVec = np.array([doseVec])

    output = np.zeros([doseVec.size, 2])

    for i, dose in enumerate(doseVec):
        output[i, :] = polyc(np.power(10, dose), np.power(10, Kx), recCount, [[1, 1]], [1.0], Cplx)[0][1:2]
    output = output[:, 0] * slopes[0] +  output[:, 0] * slopes[1]
    

    return output


In [78]:
def fitFunc():
    "Runs least squares fitting for various model parameters, and returns the minimizers"
    x0 = np.array([-12, 1, 0.5, 1, 0.5, 8.6, 8, 8, 7.6, 7, 7, 9.08, 8, 8, 8.59, 7, 7])  # KXSTAR, slopeT1h, slopeT2h, slopeT1h, slopeT2h, mIL4-IL4Ra, mIL4-Gamma, mIL4-IL13Ra, mNeo4-IL4Ra, mNeo4-Gamma, mNeo4-IL13Ra, hIL4-IL4Ra, hIL4-Gamma, hIL4-IL13Ra, hNeo4-IL4Ra, hNeo4-Gamma, hNeo4-IL13Ra (Log 10)
    bnds = ((-14, -10), (0, 10), (0, 10), (0, 10), (0, 10), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11), (4, 11))
    parampredicts = minimize(resids, x0, jac="3-point", bounds=bnds, args=[False])
    assert parampredicts.success
    return parampredicts.x

def resids(x, retDF=False):
    """"Returns residuals against signaling data"""
    masterSTAT = pd.DataFrame(columns={"Cell", "Ligand", "Concentration", "Animal", "Experimental", "Predicted"})
    Kx = x[0]
    slopeT1m = x[1]
    slopeT2m = x[2]
    slopeT1h = x[3]
    slopeT2h = x[4]
    CplxDict = {"mIL4": [[np.power(10, x[5]), 1e2, 1e2], [1e2, np.power(10, x[6]), np.power(10, x[7])]],
    "mNeo4": [[np.power(10, x[8]), 1e2, 1e2], [1e2, np.power(10, x[9]), np.power(10, x[10])]],
    "hIL4": [[np.power(10, x[11]), 1e2, 1e2], [1e2, np.power(10, x[12]), np.power(10, x[13])]],
    "hNeo4": [[np.power(10, x[14]), 1e2, 1e2], [1e2, np.power(10, x[15]), np.power(10, x[16])]]}
    SigData = pd.read_csv(join(path_here, "data/SignalingData.csv"))

    for cell in SigData.Cell.unique():
        for animal in SigData.loc[SigData.Cell == cell].Animal.unique():
            for ligand in SigData.loc[(SigData.Cell == cell) & (SigData.Animal == animal)].Ligand.unique():
                isoData = SigData.loc[(SigData.Cell == cell) & (SigData.Animal == animal) & (SigData.Ligand == ligand)]
                Concs = isoData.Concentration.values
                normSigs = isoData.Signal.values
                ligCplx = CplxDict[ligand]
                if animal == "Human":
                    results = cytBindingModel(Kx, ligCplx, Concs, cell, animal, [slopeT1h, slopeT2h])
                else:
                    results = cytBindingModel(Kx, ligCplx, Concs, cell, animal, [slopeT1m, slopeT2m])
                masterSTAT = masterSTAT.append(pd.DataFrame({"Cell": cell, "Ligand": ligand, "Concentration": Concs, "Animal": animal, "Experimental": normSigs, "Predicted": results}))
            masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal), "Predicted"] /= masterSTAT.loc[(masterSTAT.Cell == cell) & (masterSTAT.Animal == animal)].Predicted.max()
 
    if retDF[0]:
        return masterSTAT
    else:
        return np.linalg.norm(masterSTAT.Predicted.values - masterSTAT.Experimental.values)

In [79]:
fitFunc()

Ramos
[ 2949.67 16713.67     0.  ]
Ramos
[ 2949.67 16713.67     0.  ]
Macrophage
[6.651000e+01 9.804231e+04 5.837700e+02]
Macrophage
[6.651000e+01 9.804231e+04 5.837700e+02]
<ipython-input-77-baf81bfd11f1>:48: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Rtot = np.array(Rtot, dtype=np.float)
Macrophage
[17160.62 22375.29 14604.02]
Macrophage
[17160.62 22375.29 14604.02]
A549
[  133.33 18416.67  1688.  ]
A549
[  133.33 18416.67  1688.  ]
3T3
[11264.13  4190.96 15311.82]
3T3
[11264.13  4190.96 15311.82]
Ramos
[ 2949.67 16713.67     0.  ]
Ramos
[ 2949.67 16713.67     0.  ]
Macrophage
[6.651000e+01 9.804231e+04 5.837700e+02]
Macrophage
[6.651000e+01 9.804231e+04 5.

KeyboardInterrupt: 